In [1]:
from  modelclass  import model  
import modelwidget_input as mw 
from modelwidget_input import sumslidewidget, slidewidget, tabwidget, updatewidget

In [2]:
mport,baseline = model.modelload('model/port',run=1)

Zipped file read:  model\port.pcim


In [3]:
var_names = mport['new_issue_sh*'].names
def build_sorted_desc_dict(var_names):
    parsed = []

    for v in var_names:
        tail = v.split('__')[1]          # e.g. '10_YEAR_DOM'
        parts = tail.split('_')          # ['10', 'YEAR', 'DOM']

        maturity = int(parts[0])        # numeric for sorting
        currency = parts[2].lower()     # dom or usd

        currency_label = 'domestic' if currency == 'dom' else currency.upper()
        description = f"{maturity} year {currency_label} bond"

        # sort key: domestic first (0), then usd (1), then maturity
        currency_order = 0 if currency == 'dom' else 1

        parsed.append((currency_order, maturity, v, description))

    # Sort by currency first, then maturity
    parsed.sort()

    # Build ordered dict (normal dict preserves order in Python 3.7+)
    return {v: desc for _, _, v, desc in parsed}


sharedict = build_sorted_desc_dict(var_names)

In [4]:
sharedef = {d: 
            {'var':v,
            'value' : float(baseline.loc[2026,v  ]),
             'min':0,
             'max':100.0 ,'step':5.0
            } for v,d in sharedict.items()}
wsharedef =['sumslide', {'content':sharedef,'maxsum':100.0,'heading':'Issuance policy, shares of each bond'}]


In [5]:
var_names = mport['interest_rate__? interest_rate__?? interest_rate_???__? interest_rate_???__?? '].names
def build_sorted_desc_dict_implicit_dom(var_names):
    parsed = []

    for v in var_names:
        left, maturity = v.rsplit('__', 1)
        maturity = int(maturity)

        # Currency detection
        if left.endswith('_USD'):
            currency = 'usd'
            currency_label = 'USD'
        else:
            currency = 'dom'
            currency_label = 'domestic'

        description = f"{maturity} year {currency_label} bond interest rate"

        # Sort key: domestic first, then maturity
        currency_order = 0 if currency == 'dom' else 1

        parsed.append((currency_order, maturity, v, description))

    parsed.sort()

    return {v: desc for _, _, v, desc in parsed}


ratedict = build_sorted_desc_dict_implicit_dom(var_names)

In [6]:
ratedef = {d: 
            {'var':v,
            'value' : float(baseline.loc[2026,v  ]),
             'min':0,
             'max':10.0 ,'step':0.1
            } for v,d in ratedict.items()}
wratedef = ['slide',{'content':ratedef,'maxsum':100.0,'heading':'Interest rate'}]


In [7]:
portdef = ['tab',{'content':[
                              ['Markets',wratedef],
                              ['Issuance' ,wsharedef]
                            ]
                            ,'tab':True}]


wport=mw. make_widget(portdef)
#display(wport.datawidget)

In [8]:
varpat = ' stock_ultimo__all INTEREST_PAYMENTS__ALL interest_rate__? interest_rate__?? interest_rate__*26'
updatewidget(mport,wport,varpat=varpat)